In [20]:
#%pip install pandas
#%pip install psycopg2-binary

import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [21]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'dbname': 'dev',
    'user': 'adm',
    'password': 'adm'
}

In [22]:
def get_connection():
    """
    Create and return a PostgreSQL connection
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        conn.autocommit = True
        return conn
    except Exception as e:
        print(f"❌ Connection error: {e}")
        return None

In [23]:
def execute_query(query, fetch=False):
    """
    Execute a query and optionally fetch results
    """
    conn = get_connection()
    if not conn:
        return None
    
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        
        if fetch:
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            return pd.DataFrame(results, columns=columns)
        else:
            print("✅ Query executed successfully")
            return True
    except Exception as e:
        print(f"❌ Query error: {e}")
        return None
    finally:
        conn.close()

In [24]:
conn = get_connection()
if conn:
    print("✅ Database connection successful")
    conn.close()
else:
    print("❌ Database connection failed")

✅ Database connection successful


In [40]:
ddl_script = """
DROP TABLE IF EXISTS agent_state_interval CASCADE;
DROP TABLE IF EXISTS agent_event CASCADE;
DROP TABLE IF EXISTS domain_agent_states CASCADE;

CREATE TABLE domain_agent_states (
    domain_id BIGINT NOT NULL,
    state VARCHAR(200) NOT NULL,
    db_date TIMESTAMP WITHOUT TIME ZONE NOT NULL DEFAULT (NOW() AT TIME ZONE 'UTC'),
    CONSTRAINT pk_domain_agent_states PRIMARY KEY (domain_id, state)
);

-- Create index for performance
CREATE INDEX idx_domain_agent_states_domain_id ON domain_agent_states(domain_id);

CREATE TABLE agent_event (
    agent_id BIGINT NOT NULL,
    domain_id BIGINT NOT NULL,
    event_id VARCHAR(200) NOT NULL,
    timestamp_millisecond TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    agent VARCHAR(150) NOT NULL,
    state VARCHAR(40) NOT NULL,
    state_start_datetime TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    state_end_datetime TIMESTAMP WITHOUT TIME ZONE,
    dbdatetime TIMESTAMP WITHOUT TIME ZONE NOT NULL DEFAULT (NOW() AT TIME ZONE 'UTC'),
    CONSTRAINT pk_agent_event PRIMARY KEY (event_id)
);

-- Create indexes for performance
CREATE INDEX idx_agent_event_agent_domain ON agent_event(agent_id, domain_id);
CREATE INDEX idx_agent_event_state_times ON agent_event(domain_id, state_start_datetime, state_end_datetime);
CREATE INDEX idx_agent_event_timestamp ON agent_event(timestamp_millisecond);

CREATE TABLE agent_state_interval (
    agent_id BIGINT NOT NULL,
    domain_id BIGINT NOT NULL,
    interval TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    state VARCHAR(40) NOT NULL,
    agent_state_time BIGINT NOT NULL DEFAULT 0,
    db_updated_datetime TIMESTAMP WITHOUT TIME ZONE NOT NULL DEFAULT (NOW() AT TIME ZONE 'UTC'),
    db_created_datetime TIMESTAMP WITHOUT TIME ZONE NOT NULL DEFAULT (NOW() AT TIME ZONE 'UTC'),
    CONSTRAINT pk_agent_state_interval PRIMARY KEY (agent_id, domain_id, interval, state)
);

-- Create indexes for performance
CREATE INDEX idx_agent_state_interval_agent ON agent_state_interval(agent_id);
CREATE INDEX idx_agent_state_interval_interval ON agent_state_interval(interval);
CREATE INDEX idx_agent_state_interval_domain_interval ON agent_state_interval(domain_id, interval);
"""

In [48]:
# Execute DDL script
statements = [stmt.strip() for stmt in ddl_script.split(';') if stmt.strip()]
conn = get_connection()
if conn:
    cursor = conn.cursor()
    for i, statement in enumerate(statements):
        if statement and not statement.startswith('--'):
            try:
                cursor.execute(statement)
            except Exception as e:
                print(f"❌ Error in statement {i+1}: {e}")
    conn.close()
    print("🎉 DDL script execution completed")

🎉 DDL script execution completed


In [49]:
ddl_insert_script = """
INSERT INTO domain_agent_states (domain_id, state, db_date) VALUES
(1, 'NOT_READY', '2024-01-01 00:00:00'),
(1, 'READY', '2024-01-01 00:00:00'),
(1, 'ACW', '2024-01-01 00:00:00'),
(1, 'LOGGED_IN', '2024-01-01 00:00:00'),
(1, 'LOGGED_OUT', '2024-01-01 00:00:00'),
(1, 'ON_HOLD', '2024-01-01 00:00:00'),
(1, 'ON_PARK', '2024-01-01 00:00:00'),
(1, 'ON_CALL', '2024-01-01 00:00:00');

INSERT INTO agent_event (agent_id, domain_id, event_id, timestamp_millisecond, agent, state, state_start_datetime, state_end_datetime, dbdatetime) VALUES
-- Day 1 - Agent 1001
(1001, 1, 'EVT_001', '2024-09-03 08:00:00', 'AGENT_1001', 'LOGGED_IN', '2024-09-03 08:00:00', '2024-09-03 08:00:30', '2024-09-03 08:00:00'),
(1001, 1, 'EVT_002', '2024-09-03 08:00:30', 'AGENT_1001', 'NOT_READY', '2024-09-03 08:00:30', '2024-09-03 08:05:00', '2024-09-03 08:00:30'),
(1001, 1, 'EVT_003', '2024-09-03 08:05:00', 'AGENT_1001', 'READY', '2024-09-03 08:05:00', '2024-09-03 08:12:30', '2024-09-03 08:05:00'),
(1001, 1, 'EVT_004', '2024-09-03 08:12:30', 'AGENT_1001', 'ON_CALL', '2024-09-03 08:12:30', '2024-09-03 08:18:45', '2024-09-03 08:12:30'),
(1001, 1, 'EVT_005', '2024-09-03 08:18:45', 'AGENT_1001', 'ACW', '2024-09-03 08:18:45', '2024-09-03 08:20:00', '2024-09-03 08:18:45'),
(1001, 1, 'EVT_006', '2024-09-03 08:20:00', 'AGENT_1001', 'READY', '2024-09-03 08:20:00', '2024-09-03 08:25:15', '2024-09-03 08:20:00'),
(1001, 1, 'EVT_007', '2024-09-03 08:25:15', 'AGENT_1001', 'ON_CALL', '2024-09-03 08:25:15', '2024-09-03 08:35:30', '2024-09-03 08:25:15'),
(1001, 1, 'EVT_008', '2024-09-03 08:35:30', 'AGENT_1001', 'ON_HOLD', '2024-09-03 08:35:30', '2024-09-03 08:37:00', '2024-09-03 08:35:30'),
(1001, 1, 'EVT_009', '2024-09-03 08:37:00', 'AGENT_1001', 'ON_CALL', '2024-09-03 08:37:00', '2024-09-03 08:42:15', '2024-09-03 08:37:00'),
(1001, 1, 'EVT_010', '2024-09-03 08:42:15', 'AGENT_1001', 'ACW', '2024-09-03 08:42:15', '2024-09-03 08:45:00', '2024-09-03 08:42:15'),
(1001, 1, 'EVT_011', '2024-09-03 08:45:00', 'AGENT_1001', 'READY', '2024-09-03 08:45:00', '2024-09-03 09:00:00', '2024-09-03 08:45:00'),
-- Crossing 15-minute boundary (08:45 to 09:00 and beyond)
(1001, 1, 'EVT_012', '2024-09-03 09:00:00', 'AGENT_1001', 'ON_CALL', '2024-09-03 09:00:00', '2024-09-03 09:12:30', '2024-09-03 09:00:00'),
(1001, 1, 'EVT_013', '2024-09-03 09:12:30', 'AGENT_1001', 'ON_PARK', '2024-09-03 09:12:30', '2024-09-03 09:15:45', '2024-09-03 09:12:30'),
(1001, 1, 'EVT_014', '2024-09-03 09:15:45', 'AGENT_1001', 'READY', '2024-09-03 09:15:45', '2024-09-03 12:00:00', '2024-09-03 09:15:45'),
(1001, 1, 'EVT_015', '2024-09-03 12:00:00', 'AGENT_1001', 'NOT_READY', '2024-09-03 12:00:00', '2024-09-03 13:00:00', '2024-09-03 12:00:00'), -- Lunch break
(1001, 1, 'EVT_016', '2024-09-03 13:00:00', 'AGENT_1001', 'READY', '2024-09-03 13:00:00', '2024-09-03 17:00:00', '2024-09-03 13:00:00'),
(1001, 1, 'EVT_017', '2024-09-03 17:00:00', 'AGENT_1001', 'LOGGED_OUT', '2024-09-03 17:00:00', NULL, '2024-09-03 17:00:00'),

-- Day 2 - Agent 1001 (crossing day boundary)
(1001, 1, 'EVT_018', '2024-09-04 08:00:00', 'AGENT_1001', 'LOGGED_IN', '2024-09-04 08:00:00', '2024-09-04 08:00:15', '2024-09-04 08:00:00'),
(1001, 1, 'EVT_019', '2024-09-04 08:00:15', 'AGENT_1001', 'READY', '2024-09-04 08:00:15', '2024-09-04 12:00:00', '2024-09-04 08:00:15'),

-- Agent 1002: Different shift pattern
(1002, 1, 'EVT_020', '2024-09-03 09:00:00', 'AGENT_1002', 'LOGGED_IN', '2024-09-03 09:00:00', '2024-09-03 09:00:10', '2024-09-03 09:00:00'),
(1002, 1, 'EVT_021', '2024-09-03 09:00:10', 'AGENT_1002', 'READY', '2024-09-03 09:00:10', '2024-09-03 09:08:30', '2024-09-03 09:00:10'),
(1002, 1, 'EVT_022', '2024-09-03 09:08:30', 'AGENT_1002', 'ON_CALL', '2024-09-03 09:08:30', '2024-09-03 09:22:15', '2024-09-03 09:08:30'), -- Crosses 15-min boundary
(1002, 1, 'EVT_023', '2024-09-03 09:22:15', 'AGENT_1002', 'ACW', '2024-09-03 09:22:15', '2024-09-03 09:25:00', '2024-09-03 09:22:15'),
(1002, 1, 'EVT_024', '2024-09-03 09:25:00', 'AGENT_1002', 'NOT_READY', '2024-09-03 09:25:00', '2024-09-03 09:35:30', '2024-09-03 09:25:00'),
(1002, 1, 'EVT_025', '2024-09-03 09:35:30', 'AGENT_1002', 'READY', '2024-09-03 09:35:30', '2024-09-03 18:00:00', '2024-09-03 09:35:30'),
(1002, 1, 'EVT_026', '2024-09-03 18:00:00', 'AGENT_1002', 'LOGGED_OUT', '2024-09-03 18:00:00', NULL, '2024-09-03 18:00:00'),

-- Agent 1003: Night shift with state transitions
(1003, 1, 'EVT_027', '2024-09-03 22:00:00', 'AGENT_1003', 'LOGGED_IN', '2024-09-03 22:00:00', '2024-09-03 22:00:05', '2024-09-03 22:00:00'),
(1003, 1, 'EVT_028', '2024-09-03 22:00:05', 'AGENT_1003', 'READY', '2024-09-03 22:00:05', '2024-09-03 22:10:30', '2024-09-03 22:00:05'),
(1003, 1, 'EVT_029', '2024-09-03 22:10:30', 'AGENT_1003', 'ON_CALL', '2024-09-03 22:10:30', '2024-09-03 22:25:45', '2024-09-03 22:10:30'),
(1003, 1, 'EVT_030', '2024-09-03 22:25:45', 'AGENT_1003', 'ON_HOLD', '2024-09-03 22:25:45', '2024-09-03 22:28:00', '2024-09-03 22:25:45'),
(1003, 1, 'EVT_031', '2024-09-03 22:28:00', 'AGENT_1003', 'ON_CALL', '2024-09-03 22:28:00', '2024-09-03 22:35:15', '2024-09-03 22:28:00'),
(1003, 1, 'EVT_032', '2024-09-03 22:35:15', 'AGENT_1003', 'ACW', '2024-09-03 22:35:15', '2024-09-03 22:38:00', '2024-09-03 22:35:15'),
(1003, 1, 'EVT_033', '2024-09-03 22:38:00', 'AGENT_1003', 'READY', '2024-09-03 22:38:00', '2024-09-04 02:00:00', '2024-09-03 22:38:00'), -- Crosses midnight
(1003, 1, 'EVT_034', '2024-09-04 02:00:00', 'AGENT_1003', 'NOT_READY', '2024-09-04 02:00:00', '2024-09-04 02:30:00', '2024-09-04 02:00:00'), -- Break
(1003, 1, 'EVT_035', '2024-09-04 02:30:00', 'AGENT_1003', 'READY', '2024-09-04 02:30:00', '2024-09-04 06:00:00', '2024-09-04 02:30:00'),
(1003, 1, 'EVT_036', '2024-09-04 06:00:00', 'AGENT_1003', 'LOGGED_OUT', '2024-09-04 06:00:00', NULL, '2024-09-04 06:00:00');
"""

In [50]:
# Execute DDL script
statements = [stmt.strip() for stmt in ddl_insert_script.split(';') if stmt.strip()]
conn = get_connection()
if conn:
    cursor = conn.cursor()
    for i, statement in enumerate(statements):
        if statement and not statement.startswith('--'):
            try:
                cursor.execute(statement)
            except Exception as e:
                print(f"❌ Error in statement {i+1}: {e}")
    conn.close()
    print("🎉 DDL script execution completed")

🎉 DDL script execution completed


In [36]:
tables_query = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public' 
AND table_name IN ('domain_agent_states', 'agent_event', 'agent_state_interval')
ORDER BY table_name;
"""
tables_df = execute_query(tables_query, fetch=True)
print("📋 Created tables:")
display(tables_df)

📋 Created tables:


,table_name
0,agent_event
1,agent_state_interval
2,domain_agent_states


In [51]:
domain_states_df = execute_query("SELECT * FROM domain_agent_states ORDER BY state", fetch=True)
print("\n📊 Domain Agent States:")
display(domain_states_df)


📊 Domain Agent States:


,domain_id,state,db_date
0,1,ACW,2024-01-01
1,1,LOGGED_IN,2024-01-01
2,1,LOGGED_OUT,2024-01-01
3,1,NOT_READY,2024-01-01
4,1,ON_CALL,2024-01-01
5,1,ON_HOLD,2024-01-01
6,1,ON_PARK,2024-01-01
7,1,READY,2024-01-01


In [52]:
events_summary_query = """
SELECT 
    agent_id,
    COUNT(*) as event_count,
    MIN(state_start_datetime) as first_event,
    MAX(COALESCE(state_end_datetime, state_start_datetime)) as last_event,
    COUNT(DISTINCT state) as unique_states
FROM agent_event 
GROUP BY agent_id 
ORDER BY agent_id;
"""
events_summary_df = execute_query(events_summary_query, fetch=True)
print("\n📊 Agent Events Summary:")
display(events_summary_df)


📊 Agent Events Summary:


,agent_id,event_count,first_event,last_event,unique_states
0,1001,19,2024-09-03 08:00:00,2024-09-04 12:00:00,8
1,1002,7,2024-09-03 09:00:00,2024-09-03 18:00:00,6
2,1003,10,2024-09-03 22:00:00,2024-09-04 06:00:00,7
